<a href="https://colab.research.google.com/github/antmaio/Creactif_Fastai/blob/main/Fastai_Seance6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Données en Tableau

In [2]:
from fastai.tabular.all import *

In [3]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl')]

Visualisation du tableau (DataFrame) :

In [4]:
df = pd.read_csv(path/'adult.csv')
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


NaN = Not a Number ⇒ Donnée non récoltée

## Exercice 1:
Afficher les 10 derniers éléments du tableau

In [5]:
# Votre code

## Création du DataLoader

y_names : ce que l'on veut estimer (sortie du réseau de neurones)

cat_names : les entrées que l'on peut diviser en catégories (discrètes), ex: chat / chien

cont_names : les entrées qui peuvent prendre des valeurs continues, ex: valeur des pixels dans une image

In [6]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
cont_names = ['age', 'fnlwgt', 'education-num']

# dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
#                                   cat_names=cat_names, cont_names=cont_names)

Erreur car la fonction dataloaders ne comprend pas les données non numériques (idem que chat/chien)

⇒ On utilise la Transform "Categorify":

In [7]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = Categorify)
test = pd.DataFrame(dls.train_ds[['occupation','education-num']])
test = test.sort_index()
print(test[:5])

   occupation  education-num
0           0           12.0
1           5           14.0
2           0            NaN
3          11           15.0
4           9            NaN


## Exercice 2:
Pourquoi nous n'avons pas les mêmes indices que dans df.head(5) ? Comment les catégories sont-elles définies dans Categorify ?

Composition d'un batch : [entrées catégorielles, entrées continues, sorties]

In [8]:
print(f'length: {len(dls.one_batch())}')
print(dls.one_batch()[0][:5])
print(dls.one_batch()[1][:5])
print(dls.one_batch()[2][:5])

length: 3
tensor([[ 5, 13,  3,  5,  6,  5,  1],
        [ 7, 13,  3, 11,  1,  5,  2],
        [ 7, 10,  3, 11,  1,  5,  2],
        [ 7, 12,  3,  4,  1,  5,  2],
        [ 5, 12,  1,  9,  2,  5,  2]])
tensor([[6.0000e+01, 2.1669e+05, 1.0000e+01],
        [2.9000e+01, 1.1203e+05, 9.0000e+00],
        [1.7000e+01, 1.0847e+05, 7.0000e+00],
        [4.5000e+01, 2.0047e+05, 9.0000e+00],
        [3.5000e+01, 1.7797e+05, 9.0000e+00]])
tensor([[0],
        [0],
        [1],
        [0],
        [0]], dtype=torch.int8)


Pour gérer les éléments manquants, on utilise la Transform "FillMissing"

Note : les éléments manquants vont être remplacés par la valeur médiane

In [9]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = [Categorify, FillMissing])
test = pd.DataFrame(dls.train_ds[['occupation','education-num']])
test = test.sort_index()
print(test[:5])

   occupation  education-num
0           0           12.0
2           0           10.0
3          11           15.0
4           9           10.0
5           7            9.0


Pour optimiser le traitement des entrées continues, on utilise la Transform "Normalize"

In [10]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = [Categorify, FillMissing, Normalize])
test = pd.DataFrame(dls.train_ds[['occupation','education-num']])
test = test.sort_index()
print(test[:5])

   occupation  education-num
0           0       0.751136
1           5       1.535326
2           0      -0.033055
3          11       1.927421
4           9      -0.033055


In [32]:
from fastai.tabular.model import TabularModel
# print(mymodel)

mylearn = tabular_learner(dls)
mylearn.model

emb_szs = get_emb_sz(dls.train_ds, {})
emb_szs
n_cont = len(dls.cont_names)
n_out = get_c(dls)
mymodel = TabularModel(emb_szs=emb_szs, n_cont=n_cont, out_sz=n_out, layers=[45,200,100,2])
test = Learner(dls, mymodel)
test.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
    (7): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=45, out_features=45, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=45, out_features=200, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=200, out_features=100, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affin

In [33]:
test.fit(1)

epoch,train_loss,valid_loss,time
0,0.389718,0.370080,00:04
